<a href="https://colab.research.google.com/github/HKang42/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Harrison_Kang_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
df.head(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019


# Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [0]:
data = df [ (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)]

# Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.


In [7]:
data['SALE_DATE'].value_counts().sort_values()

03/30/2019     1
03/17/2019     1
04/14/2019     1
03/09/2019     1
02/17/2019     1
              ..
01/24/2019    56
01/15/2019    57
02/28/2019    58
03/29/2019    62
01/31/2019    78
Name: SALE_DATE, Length: 91, dtype: int64

The earliest date is febuary 17th 2019. So we only need to use the march 2019 cutoff

In [0]:
#pd.to_datetime( data['SALE_DATE'])

train = data[ data['SALE_DATE'] < '03/01/2019']
test = data[ data['SALE_DATE'] > '03/01/2019']

x_train = train.drop(columns=['SALE_PRICE'])
y_train = train['SALE_PRICE']

x_test = test.drop(columns=['SALE_PRICE'])
y_test = test['SALE_PRICE']

In [9]:
# Let's make sure there are no values after march for the training data

# won't use this because I don't need to see every column
#train.sort_values(by = ['SALE_DATE'])

train['SALE_DATE'].value_counts().index.sort_values()

Index(['01/01/2019', '01/02/2019', '01/03/2019', '01/04/2019', '01/07/2019',
       '01/08/2019', '01/09/2019', '01/10/2019', '01/11/2019', '01/14/2019',
       '01/15/2019', '01/16/2019', '01/17/2019', '01/18/2019', '01/21/2019',
       '01/22/2019', '01/23/2019', '01/24/2019', '01/25/2019', '01/28/2019',
       '01/29/2019', '01/30/2019', '01/31/2019', '02/01/2019', '02/04/2019',
       '02/05/2019', '02/06/2019', '02/07/2019', '02/08/2019', '02/11/2019',
       '02/12/2019', '02/13/2019', '02/14/2019', '02/15/2019', '02/17/2019',
       '02/18/2019', '02/19/2019', '02/20/2019', '02/21/2019', '02/22/2019',
       '02/25/2019', '02/26/2019', '02/27/2019', '02/28/2019'],
      dtype='object')

# Do one-hot encoding of categorical features.

In [0]:
# seems like this method for checking columns isn't going to work.....
'''
data.select_dtypes('category')
'''

In [13]:
# Let's loop through all of the features and see how many unique values are in each feature

for col in train.columns:
  values = len( data[col].value_counts())
  print( "{:<30}{:>10}".format(col , values) )

BOROUGH                                5
NEIGHBORHOOD                           7
BUILDING_CLASS_CATEGORY                1
TAX_CLASS_AT_PRESENT                   2
BLOCK                               2496
LOT                                  332
EASE-MENT                              0
BUILDING_CLASS_AT_PRESENT             13
ADDRESS                             3135
APARTMENT_NUMBER                       1
ZIP_CODE                             125
RESIDENTIAL_UNITS                      3
COMMERCIAL_UNITS                       3
TOTAL_UNITS                            4
LAND_SQUARE_FEET                    1035
GROSS_SQUARE_FEET                   1050
YEAR_BUILT                            89
TAX_CLASS_AT_TIME_OF_SALE              1
BUILDING_CLASS_AT_TIME_OF_SALE        11
SALE_PRICE                          1000
SALE_DATE                             91


I'm going to arbitrarily say that the cutoff for a categorical variable is between 2 and 6 unique values.

In [14]:
# let's see what our categorical variables are.
for col in x_train.columns:
  values = len( x_train[col].value_counts())
  if values >=2 and values <=6:
    print( "{:<30}{:>10}".format(col , values) )

BOROUGH                                4
TAX_CLASS_AT_PRESENT                   2
RESIDENTIAL_UNITS                      2
COMMERCIAL_UNITS                       3
TOTAL_UNITS                            4


In [0]:
# Now we encode
# I'm only going to use a few of the columns to minimize the number of features

import category_encoders as ce

x_train = x_train[['BOROUGH', 'NEIGHBORHOOD', 'RESIDENTIAL_UNITS']]

encoder = ce.OneHotEncoder( use_cat_names=True )
x_train = encoder.fit_transform(x_train)


x_test = x_test[['BOROUGH', 'NEIGHBORHOOD', 'RESIDENTIAL_UNITS']]
x_test = encoder.fit_transform(x_test)

In [56]:
x_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,RESIDENTIAL_UNITS
44,1,0,0,0,1,0,0,0,0,0,0,1.0
61,0,1,0,0,1,0,0,0,0,0,0,1.0
78,0,0,1,0,1,0,0,0,0,0,0,1.0
108,1,0,0,0,1,0,0,0,0,0,0,1.0
111,1,0,0,0,1,0,0,0,0,0,0,1.0


In [17]:
x_train.shape

(1708, 12)

In [18]:
x_test.shape

(1403, 13)

In [28]:
# our test data has 3 rows with borough 1 while our train data has none

x_test['BOROUGH_1'].value_counts()

0    1400
1       3
Name: BOROUGH_1, dtype: int64

In [0]:
# I'm just going to drop the borough 1 row

x_test = x_test.drop(columns=['BOROUGH_1'])

# Do feature selection with SelectKBest.

In [0]:
# run select k best on our training feature dataframe

from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func = f_regression, k = 10)

x_train_selected = selector.fit_transform(x_train, y_train)

In [33]:
# take the output array and use it to see what our best features were

selected_mask = selector.get_support()
all_names = x_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
RESIDENTIAL_UNITS


Features not selected:
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT


In [34]:
# use loop to determine best features
'''
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(x_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    x_train_selected = selector.fit_transform(x_train, y_train)
    x_test_selected = selector.transform(x_test)

    model = LinearRegression()
    model.fit(x_train_selected, y_train)
    y_pred = model.predict(x_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

'''

"\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_absolute_error\n\nfor k in range(1, len(x_train.columns)+1):\n    print(f'{k} features')\n    \n    selector = SelectKBest(score_func=f_regression, k=k)\n    x_train_selected = selector.fit_transform(x_train, y_train)\n    x_test_selected = selector.transform(x_test)\n\n    model = LinearRegression()\n    model.fit(x_train_selected, y_train)\n    y_pred = model.predict(x_test_selected)\n    mae = mean_absolute_error(y_test, y_pred)\n    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')\n\n"

# Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

In [35]:
# create ridge regression model using training data

from sklearn.linear_model import Ridge

ridge_model = Ridge( normalize=True)
ridge_model.fit(x_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

# Get mean absolute error for the test set.

In [0]:
# get our predictions

y_pred = ridge_model.predict(x_test)

In [39]:
# now let's calculate our error 

from sklearn.metrics import mean_absolute_error

print('Our mean absolute error is:', mean_absolute_error(y_test, y_pred))

Our mean absolute error is: 220456.19453864903
